In [6]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path
import time
from nozzle.util import Abi

import time
import pandas as pd

client_url = "grpc+tcp://34.122.177.97:80"
client = Client(client_url)

arbitrum_client_url = "grpc://35.223.111.195:80"
arbitrum_client = Client(arbitrum_client_url)

def elapsed(start):
    return round(time.time() - start, 4) 
def process_query(client, query):
    # df = pd.DataFrame()

    print(query)
    start = time.time()

    result_stream = client.get_sql(query)
    print('time to establish stream: ', elapsed(start), 's')

    total_events = 0
    try:
        batch = next(result_stream)
        print('time for first batch ', elapsed(start), 's')
        total_events += batch.num_rows
        df = batch.to_pandas().map(to_hex)

        print('The type of df is ', type(df))
        
        batch_start = time.time()

        for batch in result_stream:
            total_events += batch.num_rows
            print('received batch of ', batch.num_rows, ' rows in ', elapsed(batch_start), 's')
            batch_start = time.time()
            new_df = batch.to_pandas().map(to_hex)
            # Concatenate the df dataframe to the previous dataframe
            df = pd.concat([df, new_df])

            

        print('total rows: ', total_events)
        print('total time to consume the stream: ', elapsed(start), 's')
        print('rows/s: ', total_events / elapsed(start))
        return df 

    except StopIteration:
        print("No more batches available in the result stream.")

In [7]:
# The addresses and contracts of ABIs on Mainnet
# https://thegraph.com/docs/en/network/contracts/

GNS_address = 'aDcA0dd4729c8BA3aCf3E99F3A9f471EF37b6825⁠'
GNS_path = 'abis/GNS.json'

L1Staking_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
L1Staking_path =  "abis/StakingExtension.json"

IStakingStitched_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
IStakingStitched_path = "abis/IStakingStitched.json"
IStakingStitchedAbi = Abi(IStakingStitched_path)

RewardsManager_address ='9Ac758AB77733b4150A901ebd659cbF8cB93ED66⁠'
RewardsManager_path = "abis/RewardsManager.json"

ServiceRegistry_address = 'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6' 
ServiceRegistry_path =  "abis/ServiceRegistry.json"
ServiceRegistryAbi = Abi(ServiceRegistry_path)

StakingExtension_address = 'A479c00cDa8C07bce458D7a826C7b091672EB92C'
StakingExtension_path = "abis/StakingExtension.json"


In [25]:

ServiceRegistered_sig = ServiceRegistryAbi.events["ServiceRegistered"].signature()
ServiceRegistered_query = f"""
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
             CAST(pc.ServiceRegistered['indexer']  as STRING)  as indexer_str,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
    from (
        select l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') as ServiceRegistered
        from logs l
        where l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
          and l.topic0 = evm_topic('{ServiceRegistered_sig}')
        limit 3
    ) pc
"""



In [26]:
ServiceRegistered = process_query(client, ServiceRegistered_query)


    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
             CAST(pc.ServiceRegistered['indexer']  as STRING)  as indexer_str,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
    from (
        select l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
        from logs l
        where l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
          and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
        limit 3
    ) pc



FlightInternalError: Flight returned internal error, with message: planning error: This feature is not implemented: Unsupported CAST from FixedSizeBinary(20) to Utf8. gRPC client debug context: UNKNOWN:Error received from peer ipv4:34.122.177.97:80 {created_time:"2024-06-13T08:53:28.503248-04:00", grpc_status:13, grpc_message:"planning error: This feature is not implemented: Unsupported CAST from FixedSizeBinary(20) to Utf8"}. Client context: IOError: Server never sent a data message. Detail: Internal

In [35]:
# Fetch event signature
RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()


IStakingStitchedAbi = Abi(IStakingStitched_path)

RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()

ServiceRegistered_query = f"""
    SELECT  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
    FROM (
        SELECT l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') AS ServiceRegistered
        FROM logs l
        WHERE l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
          AND l.topic0 = evm_topic('{ServiceRegistered_sig}')
    ) pc
"""

RebateCollected_query = f"""
    WITH pc AS (
        {ServiceRegistered_query}
    )
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
            sw.RebateCollected['epoch'] AS epoch,
            sw.RebateCollected['tokens'] AS tokens,
            sw.RebateCollected['protocolTax'] AS protocolTax,
            sw.RebateCollected['curationFees'] AS curationFees,
            sw.RebateCollected['queryFees'] AS queryFees,
            sw.RebateCollected['queryRebates'] AS queryRebates,
            sw.RebateCollected['delegationRewards'] AS delegationRewards,
            pc.geohash
    FROM (
        SELECT l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
        FROM logs l
        WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
          AND l.topic0 = evm_topic('{RebateCollected_sig}')
    ) sw
    INNER JOIN pc
    ON sw.RebateCollected['indexer'] = pc.indexer
    WHERE pc.geohash IS NOT NULL
    LIMIT 3
"""

try:
    RebateCollected = process_query(client, RebateCollected_query)
    print(RebateCollected_query)
except Exception as e:
    print(f"Error during query execution: {e}")
    print(RebateCollected_query)


# Debugging output
print("RebateCollected Query:")
print(RebateCollected_query)

try:
    RebateCollected = process_query(client, RebateCollected_query)
    print("Query executed successfully.")
    print(RebateCollected)
except Exception as e:
    print(f"Error during query execution: {e}")

RebateCollected = process_query(client, RebateCollected_query)
print(RebateCollected_query)



    WITH pc AS (
        
    SELECT  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
    FROM (
        SELECT l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') AS ServiceRegistered
        FROM logs l
        WHERE l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
          AND l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
    ) pc

    )
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
            sw.RebateColle

FlightInternalError: Flight returned internal error, with message: query execution error: External error: External error: evm_decode
caused by
Error during planning: failed to decode data field with 0 bytes: buffer overrun while deserializing. gRPC client debug context: UNKNOWN:Error received from peer ipv4:34.122.177.97:80 {grpc_message:"query execution error: External error: External error: evm_decode\ncaused by\nError during planning: failed to decode data field with 0 bytes: buffer overrun while deserializing", grpc_status:13, created_time:"2024-06-13T09:14:01.196101-04:00"}. Client context: OK

In [19]:
RebateCollected

In [34]:
IStakingStitchedAbi = Abi(IStakingStitched_path)

RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()

# ServiceRegistered_query for reference
ServiceRegistered_query = f"""
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{ServiceRegistered_sig}')
            LIMIT 3) pc
"""

RebateCollected_query = f"""
    WITH pc AS ({ServiceRegistered_query})
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
            sw.RebateCollected['epoch'] AS epoch,
            sw.RebateCollected['tokens'] AS tokens,
            sw.RebateCollected['protocolTax'] AS protocolTax,
            sw.RebateCollected['curationFees'] AS curationFees,
            sw.RebateCollected['queryFees'] AS queryFees,
            sw.RebateCollected['queryRebates'] AS queryRebates,
            sw.RebateCollected['delegationRewards'] AS delegationRewards,
            pc.geohash
    FROM (
        SELECT l.block_num,
               evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
        FROM logs l
        WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
          AND l.topic0 = evm_topic('{RebateCollected_sig}')
    ) sw
    LEFT JOIN pc
    ON sw.RebateCollected['indexer'] = pc.indexer
"""

res = process_query(client, RebateCollected_query)



    WITH pc AS (
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
            LIMIT 3) pc
)
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
          

In [18]:

IStakingStitchedAbi = Abi(IStakingStitched_path)

RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()

RebateCollected_query = f"""
    With pc as ({ServiceRegistered_query})
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
            sw.RebateCollected['epoch'] AS epoch,
            sw.RebateCollected['tokens'] AS tokens,
            sw.RebateCollected['protocolTax'] AS protocolTax,
            sw.RebateCollected['curationFees'] AS curationFees,
            sw.RebateCollected['queryFees'] AS queryFees,
            sw.RebateCollected['queryRebates'] AS queryRebates,
            sw.RebateCollected['delegationRewards'] AS delegationRewards,
            pc.geohash
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateCollected_sig}')
) sw
LEFT JOIN pc
ON sw.RebateCollected['indexer'] = pc.indexer
where pc.geohash is not null

"""
print(RebateCollected_query)
RebateCollected = process_query(client, RebateCollected_query)


    With pc as (

    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
            LIMIT 3) pc
)
    SELECT  sw.block_num AS block_number,
            sw.RebateCollected['assetHolder'] AS assetHolder,
            sw.RebateCollected['indexer'] AS indexer,
            sw.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            sw.RebateCollected['allocationID'] AS allocationID,
         

In [17]:
RebateCollected

,block_number,assetholder,indexer,subgraphdeploymentid,allocationid,epoch,tokens,protocoltax,curationfees,queryfees,queryrebates,delegationrewards,geohash
0,18808419,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0xa6494cc9012a39f44f8450cf2191ca3a4ff3ec948796...,0x3083b182838acf185f11a7d8dbc552219ede6514,1082,1115285724343968660419,11152857243439686604,110413286710052897381,993719580390476076434,993719580390476076434,0,None
1,18808419,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0x89120082f87340211c2625500290d4d8b0e921241195...,0xee4e75e0e9ebbb3a27d0bdb576a76fd27fe57d22,1082,652833332541969908339,6528333325419699083,0,646304999216550209256,646304999216550209256,0,None
2,18808419,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0x8e541c695f2c8ee4a1819901ddca3559aeef7ae16c3c...,0x6195d30bf58583b97b776cf5b90f325111b20691,1082,599062442525703788047,5990624425257037880,59307181810044675016,533764636290402075151,533757630053697803817,0,None
3,18808419,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0x2636062ae3adee9929bc254bce28df4fb843c13d22d2...,0xb86631fd6d31b4f6623a81d80edf2f51fcb2bcb3,1082,63026647514188643634,630266475141886436,6239638103904675719,56156742935142081479,56156742935142081479,0,None
4,18839802,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x09dc2843357fbdbea7695813872c309bd611b560,0xa6494cc9012a39f44f8450cf2191ca3a4ff3ec948796...,0x92eb93e9a32e9e0f2bc6fc8ee329e1e53c3556da,1086,37727379909124977895,377273799091249778,3735010611003372811,33615095499030355306,33615095499030355306,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,18130586,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x09dc2843357fbdbea7695813872c309bd611b560,0x47c82f054fd29cc7516ef90584cc1f9921ecd15c90ac...,0x422a93fe0053391f6556937cd7c3de8b5485d3fb,988,138796862734118037005,1387968627341180370,13740889410677685663,123668004696099170972,123668004696099170972,0,None
1,18133124,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0xa6494cc9012a39f44f8450cf2191ca3a4ff3ec948796...,0xe406f2f0feac2bdcd4218299b112276c55aa3816,988,711379579564346527699,7113795795643465276,70426578376870306242,633839205391832756181,519748148421302860068,114091056970529896113,None
2,18133124,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0xde15ba95e8bce330db8e0b5e2f28ece2d3c217294ce1...,0x68efe3372f535db034be84d637b22b7607bcfc59,988,188242658197878976207,1882426581978789762,18636023161590018644,167724208454310167801,137533850932534337596,30190357521775830205,None
3,18133124,0x4a53cf3b3eda545dc61dee0ca21ea8996c94385f,0x1a99dd7d916117a523f3ce6510dcfd6bceab11e7,0x8e541c695f2c8ee4a1819901ddca3559aeef7ae16c3c...,0x017e99732a6b957e22246826bed49adbe77fa5d5,988,177517328751045946980,1775173287510459469,17574215546353548751,158167939917181938760,129697710732089189783,28470229185092748977,None


In [12]:
RebateCollected_query = f"""
    With sr as ({ServiceRegistered_query}),
    SELECT  pc.block_num AS block_number,
            pc.RebateCollected['assetHolder'] AS assetHolder,
            pc.RebateCollected['indexer'] AS indexer,
            pc.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateCollected['allocationID'] AS allocationID,
            pc.RebateCollected['epoch'] AS epoch,
            pc.RebateCollected['tokens'] AS tokens,
            pc.RebateCollected['protocolTax'] AS protocolTax,
            pc.RebateCollected['curationFees'] AS curationFees,
            pc.RebateCollected['queryFees'] AS queryFees,
            pc.RebateCollected['queryRebates'] AS queryRebates,
            pc.RebateCollected['delegationRewards'] AS delegationRewards,
            sr.geohash
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateCollected_sig}')
) pc 
LEFT JOIN sr 
ON pc.RebateCollected['indexer'] = sr.indexer

"""

RebateCollected = process_query(client, RebateCollected_query)



    With sr as (

    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
            LIMIT 3
) pc
),
    SELECT  pc.block_num AS block_number,
            pc.RebateCollected['assetHolder'] AS assetHolder,
            pc.RebateCollected['indexer'] AS indexer,
            pc.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateCollected['allocationID'] AS allocationID,
       

ArrowInvalid: Flight returned invalid argument error, with message: unsupported SQL: SQL error: ParserError("Expected AS, found: pc")

In [112]:
ServiceRegistered 

In [ ]:

ServiceRegistered_query1 = f"""
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{ServiceRegistered_sig}')
      
) pc
"""



StakingExtensionAbi = Abi(StakingExtension_path)

StakingExtension_created_sig_1 = StakingExtensionAbi.events["StakeDelegated"].signature()
StakingExtension_created_sig_2 = StakingExtensionAbi.events["StakeDelegatedLocked"].signature()


IStakingStitchedAbi = Abi(IStakingStitched_path)

AllocationCreated_sig = IStakingStitchedAbi.events["AllocationCreated"].signature()

AllocationCreated_query = f"""
    select  pc.block_num as block_number,
            pc.AllocationCreated['metadata'] as metadata,
            pc.AllocationCreated['allocationID'] as allocationID,
            pc.AllocationCreated['tokens'] as tokens,
            pc.AllocationCreated['epoch'] as epoch,
            pc.AllocationCreated['subgraphDeploymentID'] as subgraphDeploymentID,
            pc.AllocationCreated['indexer'] as indexer,
            pc.AllocationCreated['rebateFees'] as rebateFees
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{AllocationCreated_sig}') as AllocationCreated
                from logs l
             where l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{AllocationCreated_sig}')
) pc
"""

AllocationCreated = process_query(client, AllocationCreated_query)

AllocationCollected_sig = IStakingStitchedAbi.events["AllocationCollected"].signature()

AllocationCollected_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.AllocationCollected['indexer'] AS indexer,
            pc.AllocationCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.AllocationCollected['epoch'] AS epoch,
            pc.AllocationCollected['tokens'] AS tokens,
            pc.AllocationCollected['allocationID'] AS allocationID,
            pc.AllocationCollected['from'] AS from,
            pc.AllocationCollected['curationFees'] AS curationFees,
            pc.AllocationCollected['rebateFees'] AS rebateFees
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{AllocationCollected_sig}') AS AllocationCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{AllocationCollected_sig}')
) pc
"""

AllocationCollected = process_query(client, AllocationCollected_query)



StakeDelegated_sig = IStakingStitchedAbi.events["StakeDelegated"].signature()

StakeDelegated_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDelegated['indexer'] AS indexer,
            pc.StakeDelegated['delegator'] AS delegator,
            pc.StakeDelegated['tokens'] AS tokens,
            pc.StakeDelegated['shares'] AS shares
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDelegated_sig}') AS StakeDelegated
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDelegated_sig}')
) pc
"""

StakeDelegated = process_query(client, StakeDelegated_query)

StakeDeposited_sig = IStakingStitchedAbi.events["StakeDeposited"].signature()

StakeDeposited_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDeposited['indexer'] AS indexer,
            pc.StakeDeposited['tokens'] AS tokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDeposited_sig}') AS StakeDeposited
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDeposited_sig}')
) pc
"""

StakeDeposited = process_query(client, StakeDeposited_query)

StakeLocked_sig = IStakingStitchedAbi.events["StakeLocked"].signature()

StakeLocked_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeLocked['indexer'] AS indexer,
            pc.StakeLocked['tokens'] AS tokens,
            pc.StakeLocked['until'] AS until
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeLocked_sig}') AS StakeLocked
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeLocked_sig}')
) pc
"""

StakeLocked = process_query(client, StakeLocked_query)

RebateClaimed_sig = IStakingStitchedAbi.events["RebateClaimed"].signature()

RebateClaimed_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RebateClaimed['indexer'] AS indexer,
            pc.RebateClaimed['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateClaimed['allocationID'] AS allocationID,
            pc.RebateClaimed['epoch'] AS epoch,
            pc.RebateClaimed['forEpoch'] AS forEpoch,
            pc.RebateClaimed['tokens'] AS tokens,
            pc.RebateClaimed['unclaimedAllocationsCount'] AS unclaimedAllocationsCount,
            pc.RebateClaimed['delegationFees'] AS delegationFees
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateClaimed_sig}') AS RebateClaimed
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateClaimed_sig}')
) pc
"""

RebateClaimed = process_query(client, RebateClaimed_query)

StakeDelegatedLocked_sig = IStakingStitchedAbi.events["StakeDelegatedLocked"].signature()

StakeDelegatedLocked_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDelegatedLocked['indexer'] AS indexer,
            pc.StakeDelegatedLocked['delegator'] AS delegator,
            pc.StakeDelegatedLocked['tokens'] AS tokens,
            pc.StakeDelegatedLocked['shares'] AS shares,
            pc.StakeDelegatedLocked['until'] AS until
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDelegatedLocked_sig}') AS StakeDelegatedLocked
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDelegatedLocked_sig}')
) pc
"""

StakeDelegatedLocked = process_query(client, StakeDelegatedLocked_query)

RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()

RebateCollected_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RebateCollected['assetHolder'] AS assetHolder,
            pc.RebateCollected['indexer'] AS indexer,
            pc.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateCollected['allocationID'] AS allocationID,
            pc.RebateCollected['epoch'] AS epoch,
            pc.RebateCollected['tokens'] AS tokens,
            pc.RebateCollected['protocolTax'] AS protocolTax,
            pc.RebateCollected['curationFees'] AS curationFees,
            pc.RebateCollected['queryFees'] AS queryFees,
            pc.RebateCollected['queryRebates'] AS queryRebates,
            pc.RebateCollected['delegationRewards'] AS delegationRewards
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateCollected_sig}')
) pc
"""

RebateCollected = process_query(client, RebateCollected_query)

StakeWithdrawn_sig = IStakingStitchedAbi.events["StakeWithdrawn"].signature()

StakeWithdrawn_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeWithdrawn['indexer'] AS indexer,
            pc.StakeWithdrawn['tokens'] AS tokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeWithdrawn_sig}') AS StakeWithdrawn
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeWithdrawn_sig}')
) pc
"""

StakeWithdrawn = process_query(client, StakeWithdrawn_query)

StakeSlashed_sig = IStakingStitchedAbi.events["StakeSlashed"].signature()

StakeSlashed_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeSlashed['indexer'] AS indexer,
            pc.StakeSlashed['tokens'] AS tokens,
            pc.StakeSlashed['reward'] AS reward,
            pc.StakeSlashed['beneficiary'] AS beneficiary
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeSlashed_sig}') AS StakeSlashed
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeSlashed_sig}')
) pc
"""

StakeSlashed = process_query(client, StakeSlashed_query)

In [17]:
L1Staking_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
L1Staking_path =  "abis/L1Staking.json"
L1StakingAbi = Abi(L1Staking_path)

IndexerStakeTransferredToL2_sig = L1StakingAbi.events["IndexerStakeTransferredToL2"].signature()

IndexerStakeTransferredToL2_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.IndexerStakeTransferredToL2['indexer'] AS indexer,
            pc.IndexerStakeTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.IndexerStakeTransferredToL2['transferredStakeTokens'] AS transferredStakeTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{IndexerStakeTransferredToL2_sig}') AS IndexerStakeTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'{L1Staking_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{IndexerStakeTransferredToL2_sig}')
) pc
"""

IndexerStakeTransferredToL2 = process_query(client, IndexerStakeTransferredToL2_query)



    SELECT  pc.block_num AS block_number,
            pc.IndexerStakeTransferredToL2['indexer'] AS indexer,
            pc.IndexerStakeTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.IndexerStakeTransferredToL2['transferredStakeTokens'] AS transferredStakeTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'IndexerStakeTransferredToL2(address indexed indexer,address indexed l2Indexer,uint256 transferredStakeTokens)') AS IndexerStakeTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'F55041E37E12cD407ad00CE2910B8269B01263b9', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('IndexerStakeTransferredToL2(address indexed indexer,address indexed l2Indexer,uint256 transferredStakeTokens)')
) pc

time to establish stream:  0.1039 s
No more batches available in the result stream.


In [18]:
DelegationTransferredToL2_sig = L1StakingAbi.events["DelegationTransferredToL2"].signature()
DelegationTransferredToL2_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.DelegationTransferredToL2['delegator'] AS delegator,
            pc.DelegationTransferredToL2['l2Delegator'] AS l2Delegator,
            pc.DelegationTransferredToL2['indexer'] AS indexer,
            pc.DelegationTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.DelegationTransferredToL2['transferredDelegationTokens'] AS transferredDelegationTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{DelegationTransferredToL2_sig}') AS DelegationTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'{L1Staking_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{DelegationTransferredToL2_sig}')
) pc
"""

DelegationTransferredToL2 = process_query(client, DelegationTransferredToL2_query)



    SELECT  pc.block_num AS block_number,
            pc.DelegationTransferredToL2['delegator'] AS delegator,
            pc.DelegationTransferredToL2['l2Delegator'] AS l2Delegator,
            pc.DelegationTransferredToL2['indexer'] AS indexer,
            pc.DelegationTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.DelegationTransferredToL2['transferredDelegationTokens'] AS transferredDelegationTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'DelegationTransferredToL2(address indexed delegator,address indexed l2Delegator,address indexed indexer,address l2Indexer,uint256 transferredDelegationTokens)') AS DelegationTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'F55041E37E12cD407ad00CE2910B8269B01263b9', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('DelegationTransferredToL2(address indexed delegator,address indexed l2Delegator,address indexed indexer,address l2

In [9]:
RewardsManager_address = '9Ac758AB77733b4150A901ebd659cbF8cB93ED66'
RewardsManager_path =  "abis/RewardsManager.json"
RewardsManagerAbi = Abi(RewardsManager_path)
RewardsAssigned_sig = RewardsManagerAbi.events["RewardsAssigned"].signature()


RewardsAssigned_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RewardsAssigned['indexer'] AS indexer,
            pc.RewardsAssigned['allocationID'] AS allocationID,
            pc.RewardsAssigned['epoch'] AS epoch,
            pc.RewardsAssigned['amount'] AS amount
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RewardsAssigned_sig}') AS RewardsAssigned
              FROM logs l
             WHERE l.address = arrow_cast(x'{RewardsManager_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RewardsAssigned_sig}')
               LIMIT 3
) pc
"""

RewardsAssigned = process_query(client, RewardsAssigned_query)



    SELECT  pc.block_num AS block_number,
            pc.RewardsAssigned['indexer'] AS indexer,
            pc.RewardsAssigned['allocationID'] AS allocationID,
            pc.RewardsAssigned['epoch'] AS epoch,
            pc.RewardsAssigned['amount'] AS amount
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'RewardsAssigned(address indexed indexer,address indexed allocationID,uint256 epoch,uint256 amount)') AS RewardsAssigned
              FROM logs l
             WHERE l.address = arrow_cast(x'9Ac758AB77733b4150A901ebd659cbF8cB93ED66', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('RewardsAssigned(address indexed indexer,address indexed allocationID,uint256 epoch,uint256 amount)')
               LIMIT 3
) pc

time to establish stream:  0.2265 s
time for first batch  83.0681 s
The type of df is  <class 'pandas.core.frame.DataFrame'>
total rows:  3
total time to consume the stream:  83.0713 s
rows/s:  0.03611355546380

To process each event in the block number order using the provided Typescript logic, we will need to:

Combine all event DataFrames into one DataFrame.
Sort the combined DataFrame by block_number.
Process each event based on its type usming the provided logic.


In [81]:

# Adding the event_type column to each DataFrame
# ServiceRegistered = ServiceRegistered.assign(event_type='ServiceRegistered')
AllocationCreated = AllocationCreated.assign(event_type='AllocationCreated')
AllocationCollected = AllocationCollected.assign(event_type='AllocationCollected')
IndexerStakeTransferredToL2 = IndexerStakeTransferredToL2.assign(event_type='IndexerStakeTransferredToL2')

StakeDelegated = StakeDelegated.assign(event_type='StakeDelegated')
StakeDeposited = StakeDeposited.assign(event_type='StakeDeposited')
StakeLocked = StakeLocked.assign(event_type='StakeLocked')
RebateClaimed = RebateClaimed.assign(event_type='RebateClaimed')
DelegationTransferredToL2 = DelegationTransferredToL2.assign(event_type='DelegationTransferredToL2')

StakeDelegatedLocked = StakeDelegatedLocked.assign(event_type='StakeDelegatedLocked')
RewardsAssigned = RewardsAssigned.assign(event_type='RewardsAssigned')
RebateCollected = RebateCollected.assign(event_type='RebateCollected')
StakeWithdrawn = StakeWithdrawn.assign(event_type='StakeWithdrawn')
StakeSlashed = StakeSlashed.assign(event_type='StakeSlashed')

# Combine all DataFrames
all_events_df = pd.concat([
    ServiceRegistered,
    AllocationCreated,
    AllocationCollected,
    IndexerStakeTransferredToL2,
    StakeDelegated,
    StakeDeposited,
    StakeLocked,
    RebateClaimed,
    DelegationTransferredToL2,
    StakeDelegatedLocked,
    RewardsAssigned,
    RebateCollected,
    StakeWithdrawn,
    StakeSlashed
])

# Sort by block_number
all_events_df = all_events_df.sort_values(by='block_number').reset_index(drop=True)


In [103]:
from collections import defaultdict

# Initialize indexerStored
indexerStored = defaultdict(lambda: {
    'stakedTokens': 0,
    'geoHash': '',
    'allocatedTokens': 0,
    'totalAllocationCount': 0,
    'allocationCount': 0,
    'queryFeesCollected': 0,
    'delegatedTokens': 0,
    'lockedTokens': 0,
    'queryFeeRebates': 0,
    'indexingRewardCut': 0,
    'tokensLockedstakedTokens': 0,
    'indexerIndexingRewards': 0
})

# Function to process events
def process_events(df, indexerStored):
    for _, row in df.iterrows():
        event_type = row['event_type']
        indexer = row['indexer']
        
        if event_type == 'ServiceRegistered':
            indexerStored[indexer]['stakedTokens'] -= row['transferredStakeTokens']
            indexerStored[indexer]['geoHash'] = row['geoHash']
        
        elif event_type == 'AllocationCreated':
            # indexerStored[indexer]['allocatedTokens'] += row['tokens']
            indexerStored[indexer]['totalAllocationCount'] += 1
            indexerStored[indexer]['allocationCount'] += 1
            # indexerStored[indexer]['queryFeesCollected'] += row['rebateFees']
        
        elif event_type == 'AllocationCollected':
            indexerStored[indexer]['queryFeesCollected'] += 0
            # indexerStored[indexer]['queryFeesCollected'] += row['rebateFees']
        
        elif event_type == 'IndexerStakeTransferredToL2':
            continue
            # indexerStored[indexer]['stakedTokens'] -= row['transferredStakeTokens']
        
        elif event_type == 'StakeDelegated':
            indexerStored[indexer]['delegatedTokens'] += row['tokens']
        
        elif event_type == 'StakeDeposited':
            indexerStored[indexer]['stakedTokens'] += row['tokens']
        
        elif event_type == 'StakeLocked':
            indexerStored[indexer]['lockedTokens'] = row['tokens']
        
        elif event_type == 'RebateClaimed':
            # indexerStored[indexer]['delegatedTokens'] += row['delegationFees']
            indexerStored[indexer]['queryFeeRebates'] += row['tokens']
        
        elif event_type == 'DelegationTransferredToL2':
            # indexerStored[indexer]['delegatedTokens'] -= row['transferredDelegationTokens']
            continue
        
        elif event_type == 'StakeDelegatedLocked':
            indexerStored[indexer]['delegatedTokens'] -= row['tokens']
        
        elif event_type == 'RewardsAssigned':
            if indexerStored[indexer]['delegatedTokens'] == 0:
                indexerStored[indexer]['indexerIndexingRewards'] = row['amount']
            else:
                indexerStored[indexer]['indexerIndexingRewards'] = row['amount'] * indexerStored[indexer]['indexingRewardCut'] // 1000000
            indexerStored[indexer]['delegatedTokens'] += indexerStored[indexer]['indexerIndexingRewards']
        
        elif event_type == 'RebateCollected':
            continue
            # indexerStored[indexer]['delegatedTokens'] += row['delegationRewards']
            # indexerStored[indexer]['queryFeesCollected'] += row['queryFees']
            # indexerStored[indexer]['queryFeeRebates'] += row['queryRebates']
        
        elif event_type == 'StakeWithdrawn':
            indexerStored[indexer]['lockedTokens'] -= row['tokens']
        
        elif event_type == 'StakeSlashed':
            indexerStored[indexer]['lockedTokens'] = indexerStored[indexer]['tokensLockedstakedTokens']
            indexerStored[indexer]['stakedTokens'] -= row['tokens']

# Process all events
process_events(all_events_df, indexerStored)

# Print the updated indexerStored
print(indexerStored)


defaultdict(<function <lambda> at 0x15c409440>, {'0x06590a641dc3eb43f2cebe435576389f209116da': {'stakedTokens': Decimal('10000000000000000000'), 'geoHash': '', 'allocatedTokens': 0, 'totalAllocationCount': 0, 'allocationCount': 0, 'queryFeesCollected': 0, 'delegatedTokens': 0, 'lockedTokens': Decimal('0'), 'queryFeeRebates': 0, 'indexingRewardCut': 0, 'tokensLockedstakedTokens': 0, 'indexerIndexingRewards': 0}, '0xf62de2a9d8b8033a5b66f497dca1bd91080316fb': {'stakedTokens': Decimal('100000000000000000000000'), 'geoHash': '', 'allocatedTokens': 0, 'totalAllocationCount': 0, 'allocationCount': 0, 'queryFeesCollected': 0, 'delegatedTokens': Decimal('0'), 'lockedTokens': Decimal('0'), 'queryFeeRebates': 0, 'indexingRewardCut': 0, 'tokensLockedstakedTokens': 0, 'indexerIndexingRewards': 0}, '0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a': {'stakedTokens': Decimal('4785367456361618270620872'), 'geoHash': '', 'allocatedTokens': 0, 'totalAllocationCount': 1574, 'allocationCount': 1574, 'queryFeesC

In [104]:

# Convert the defaultdict to a regular dictionary
indexer_dict = {key: dict(value) for key, value in indexerStored.items()}

# Convert the dictionary to a pandas DataFrame
indexer_df = pd.DataFrame.from_dict(indexer_dict, orient='index')

# Reset index to turn the index into a column
indexer_df.reset_index(inplace=True)
indexer_df.rename(columns={'index': 'indexer'}, inplace=True)

# Convert large numeric columns to string to avoid numeric range issues in Parquet
columns_to_convert = ['stakedTokens', 'allocatedTokens', 'queryFeesCollected', 'delegatedTokens', 'lockedTokens', 'queryFeeRebates', 'tokensLockedstakedTokens', 'indexerIndexingRewards']
indexer_df[columns_to_convert] = indexer_df[columns_to_convert].applymap(str)


# Save the DataFrame to a Parquet file
indexer_df.to_parquet('indexerStored.parquet', engine='pyarrow')

# Print the DataFrame to check the conversion
print(indexer_df)

                                        indexer               stakedTokens  \
0    0x06590a641dc3eb43f2cebe435576389f209116da       10000000000000000000   
1    0xf62de2a9d8b8033a5b66f497dca1bd91080316fb   100000000000000000000000   
2    0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a  4785367456361618270620872   
3    0xf3f182e859f789af8d6c2223d15c691471c79a2b  2331563819532313429219378   
4    0x163a2ebf98ef202da5834a33f7ce98098a0b7874   192308000000000000000000   
..                                          ...                        ...   
573  0xe2571c87f1433ea06be389e427af2a17bfd37fc0   210571267754876523306445   
574  0x9ada64ca2fa91318e2de0ff678a8a935859954f4   100674499853717515400000   
575  0x56598b0816dd39fea9aef8e4dbb49e87f5b10fbf   100000000000000000000000   
576  0x2d2771e17fe8daff29b6d903559ec1c8192945b0   100501767136255316115456   
577  0xd11eb5db7cfbb9ecae4b62e71ec0a461f6baf669   100000000000000000000000   

    geoHash allocatedTokens  totalAllocationCount  allocationCo

/var/folders/5t/tktsv9cs629bjbjkqylhtw100000gn/T/ipykernel_97758/1637334497.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  indexer_df[columns_to_convert] = indexer_df[columns_to_convert].applymap(str)


In [107]:
from google.cloud import storage, bigquery


def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Example usage
bucket_name = 'nozzle-data-science'
source_file_name = '/Users/vivianpeng/nozzle/project-nozzle/python/examples/the_graph/pq_data/indexerStored.parquet'
destination_blob_name = 'path/in/bucket/indexer_nozzle.parquet'

upload_to_gcs(bucket_name, source_file_name, destination_blob_name)

def load_parquet_to_bigquery(dataset_id, table_id, gcs_uri):
    """Loads a Parquet file from GCS into BigQuery."""
    client = bigquery.Client()

    table_ref = client.dataset(dataset_id).table(table_id)
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
    )

    load_job = client.load_table_from_uri(
        gcs_uri,
        table_ref,
        job_config=job_config
    )

    print(f"Starting job {load_job.job_id}")

    load_job.result()  # Waits for the job to complete.

    print(f"Job finished. Loaded {load_job.output_rows} rows into {dataset_id}:{table_id}")



/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


File /Users/vivianpeng/nozzle/project-nozzle/python/examples/the_graph/pq_data/indexerStored.parquet uploaded to path/in/bucket/indexer_nozzle.parquet.


In [108]:
# Example usage
dataset_id = 'internal_metrics'
table_id = 'indexer'
gcs_uri = f'gs://{bucket_name}/{destination_blob_name}'

load_parquet_to_bigquery(dataset_id, table_id, gcs_uri)


/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Starting job e1179b76-51c0-439c-ba1f-1e7a279bd216


BadRequest: 400 Error while reading data, error message: The value for column 'allocatedTokens' is out of valid NUMERIC range: Value is out of range after scaling to NUMERIC type; input length: 13; scale: 0; input data: 01a46d77c6247507abbfaff200. Consider specifying decimal target types to allow a larger range of values. See https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-parquet#decimal_logical_type; in column 'allocatedTokens' File: gs://nozzle-data-science/path/in/bucket/indexer_nozzle.parquet; reason: invalid, location: gs://nozzle-data-science/path/in/bucket/indexer_nozzle.parquet, message: Error while reading data, error message: The value for column 'allocatedTokens' is out of valid NUMERIC range: Value is out of range after scaling to NUMERIC type; input length: 13; scale: 0; input data: 01a46d77c6247507abbfaff200. Consider specifying decimal target types to allow a larger range of values. See https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-